In [ ]:
#default_exp filter
%load_ext autoreload
%autoreload 2

# Filtragem
> Este módulo concentra funções auxiliares específicas que filtram os dados do banco com campos e formatação de interesse para aplicações específicas como o [appAnalise](https://github.com/EricMagalhaesDelgado/appAnalise) por exemplo.

In [ ]:
#export
from pathlib import Path
import pandas as pd
from datetime import datetime
from anateldb.parser import *
from anateldb.query import *
from fastcore.test import *

In [ ]:
#export 
def filter_database(path):
    path = Path(path)
    dest = path / 'FiltroAppAnalise'
    dest.mkdir(parents = True, exist_ok=True)
    for p in dest.iterdir(): 
        if p.is_file():
            p.unlink()
    time = datetime.today().strftime("%Y%m%d_%H%M")
    mosaico = read_mosaico(path)
    stel = read_stel(path).sort_values('Frequência').loc[:, TELECOM]
    radcom = read_radcom(path)
    radcom['Serviço'] = '231'
    radcom['Status'] = 'RADCOM'
    radcom['Classe'] = radcom.Fase + '-' + radcom.Situação 
    radcom = radcom.loc[:, RADIODIFUSAO]
    mosaico = mosaico.loc[:, RADIODIFUSAO]
    radiodifusao = mosaico.append(radcom)
    radiodifusao['Frequência'] = radiodifusao['Frequência'].str.replace(',','.')
    radiodifusao['Frequência'] = radiodifusao.Frequência.astype('float32')
    radiodifusao = radiodifusao.sort_values('Frequência').reset_index(drop=True)
    slma = (stel.Frequência >= 108) & (stel.Frequência <= 137)
    telecom = stel[~slma].reset_index(drop=True)
    slma = stel[slma].reset_index(drop=True)
    radiodifusao = radiodifusao[(radiodifusao.Latitude != '') & (radiodifusao.Longitude != '')].reset_index(drop=True)
    radiodifusao['Latitude'] = radiodifusao.Latitude.str.replace(',', '.').astype('float32')
    radiodifusao['Longitude'] = radiodifusao.Longitude.str.replace(',', '.').astype('float32')
    
    for c in radiodifusao.columns:
        if c in ('Latitude', 'Longitude'):
            radiodifusao[c] = radiodifusao[c].astype('float32', errors='ignore')
        else:
            radiodifusao[c] = radiodifusao[c].astype('str')
    for c in slma.columns: 
        if c in ('Latitude', 'Longitude'):
            slma[c] = slma[c].astype('float32', errors='ignore')
        else:
            slma[c] = slma[c].astype('str')
    for c in telecom.columns: 
        if c in ('Latitude', 'Longitude'):
            telecom[c] = telecom[c].astype('float32', errors='ignore')
        else:
            telecom[c] = telecom[c].astype('str')
            
    radiodifusao['Descrição'] = radiodifusao.Status + ', ' + radiodifusao.Classe + ', ' + radiodifusao.Entidade.str.title() + ' (' + \
                                radiodifusao.Fistel + ', ' + radiodifusao['Número da Estação'] + '), ' + radiodifusao.Município + '/' + \
                                radiodifusao.UF
    slma['Descrição'] = slma.Serviço + ', ' + slma.Entidade.str.title() + ' (' + slma.Fistel + ', ' +  slma['Número da Estação'] + '), ' + \
                        slma['Município'] + '/' + slma.UF
    telecom['Descrição'] = telecom.Serviço + ', ' + telecom.Entidade.str.title() + ' (' + telecom.Fistel + ', ' + telecom['Número da Estação'] + '), ' + \
                           telecom['Município'] + '/' + telecom.UF
    
    export_columns = ['Frequência', 'Latitude', 'Longitude', 'Descrição']
    radiodifusao = radiodifusao.loc[:, export_columns]
    slma = slma.loc[:, export_columns]
    telecom = telecom.loc[:, export_columns]
    radiodifusao.to_csv(f'{dest}/radiodifusao_{time}.csv', index=False)
    slma.to_csv(f'{dest}/108-137_{time}.csv', index=False)
    telecom.to_csv(f'{dest}/telecom_{time}.csv', index=False)
    radiodifusao.to_feather(f'{dest}/radiodifusao_{time}.fth')
    slma.to_feather(f'{dest}/108-137_{time}.fth')
    telecom.to_feather(f'{dest}/telecom_{time}.fth')
    with pd.ExcelWriter(f'{dest}/Base_de_Dados_{time}.xlsx') as workbook:
        radiodifusao.to_excel(workbook, sheet_name='Radiodifusão', index=False)
        slma.to_excel(workbook, sheet_name='108-137', index=False)
        telecom.to_excel(workbook, sheet_name='Telecom', index=False)

In [ ]:
from nbdev.export import notebook2script; notebook2script()

Converted filter.ipynb.
Converted index.ipynb.
Converted parser.ipynb.
Converted queries.ipynb.
